In [2]:
#############
# LIBRARIES #
#############

import os, sys

import requests
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import pandas as pd
import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


In [8]:
########################################
# Accessing my data through Strava API #
########################################

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

payload = {
    'client_id': "36305",
    'client_secret': '81b8fd9d50c305cacba51667b8d9fbde7583abd0',
    'refresh_token': '5eabc7e0e6edd853509a148c9e0fe6dd307bfd5a',
    'grant_type': "refresh_token",
    'f': 'json'
}

print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(activites_url, headers=header, params=param).json()

print(my_dataset[0]["name"]) # Getting last activity name
# print(my_dataset[0]["map"]["summary_polyline"])

Requesting Token...

Access Token = c7838431fb42ab0af8cc8e16ebf95438a94409d0

Windy! 💨 


In [13]:
# Converting json file to Pandas DataFrame
my_dataset_df = pd.DataFrame(my_dataset)
my_dataset_df.head()


,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,workout_type,id,...,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score
0,2,"{'id': 644338, 'resource_state': 1}",Windy! 💨,30998.3,4379,4714,81.0,Ride,10.0,6937664224,...,15.0,-51.0,7.382436e+09,7382436407,karooUpload.fit,False,8,0,False,128.0
1,2,"{'id': 644338, 'resource_state': 1}",Evening Activity,0.0,1724,1724,0.0,Workout,NaN,6932749848,...,NaN,NaN,7.377172e+09,7377171859,44A43A92-D4B8-4D32-B8D2-89FC38E85481.fit,False,0,0,False,34.0
2,2,"{'id': 644338, 'resource_state': 1}",Errands,8042.2,1535,2976,7.6,Ride,10.0,6933025280,...,7.2,2.3,7.377468e+09,7377468070,77C24CEA-0D43-45EB-BD28-CAF6E315C538.fit,False,0,0,False,10.0
3,2,"{'id': 644338, 'resource_state': 1}",Riding with the boy,16471.2,3675,4071,9.8,Ride,10.0,6928412920,...,6.6,1.2,7.372481e+09,7372480657,71A436B7-9C5A-4E50-84C4-6E974242F823.fit,False,0,4,False,12.0
4,2,"{'id': 644338, 'resource_state': 1}",Tunnel to Steam Trains,53447.2,9577,10103,689.0,Ride,10.0,6922321736,...,506.6,-2.4,7.365839e+09,7365838742,karooUpload.fit,False,16,3,False,309.0


In [13]:
########################################
# Accessing my data through Strava API #
########################################

client_id = '36305'
client_secret = '81b8fd9d50c305cacba51667b8d9fbde7583abd0'
strava_code = 'f842cb19d9259fb46cc03467a15caa780695fcdc'
refresh_token = '5eabc7e0e6edd853509a148c9e0fe6dd307bfd5a'



POST_request = 'https://www.strava.com/oauth/token?client_id=36305&client_secret=81b8fd9d50c305cacba51667b8d9fbde7583abd0&code=f842cb19d9259fb46cc03467a15caa780695fcdc&grant_type=authorization_code'


In [9]:

# # Make Strava auth API call with your 
# # client_code, client_secret and code
# response = requests.post(
#                     url = 'https://www.strava.com/oauth/token',
#                     data = {
#                             'client_id': '36305',
#                             'client_secret': '81b8fd9d50c305cacba51667b8d9fbde7583abd0',
#                             'code': 'f842cb19d9259fb46cc03467a15caa780695fcdc',
#                             'grant_type': 'authorization_code'
#                             }
#                 )
# #Save json response as a variable
# strava_tokens = response.json()
# # Save tokens to file
# with open('strava_tokens.json', 'w') as outfile:
#     json.dump(strava_tokens, outfile)
# # Open JSON file and print the file contents 
# # to check it's worked properly
# with open('strava_tokens.json') as check:
#   data = json.load(check)
# print(data)

In [6]:
auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

payload = {
    'client_id': "36305",
    'client_secret': '81b8fd9d50c305cacba51667b8d9fbde7583abd0',
    'refresh_token': '5eabc7e0e6edd853509a148c9e0fe6dd307bfd5a',
    'grant_type': "refresh_token",
    'f': 'json'
}

print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(activites_url, headers=header, params=param).json()

print(my_dataset[0]["name"])
print(my_dataset[0]["map"]["summary_polyline"])

Requesting Token...

Access Token = c7838431fb42ab0af8cc8e16ebf95438a94409d0

Windy! 💨 
okseFt{hiVx@y@dBy@v@{@\}@tBsHb@{@nBgBzBaBh@QbCSh@DTK~AsF|AuAh@q@n@oB`@_EDaDQcFFk@dD@b@IZeCbAyB`DgDvByAV?`Ap@x@NhEFb@SRsEPeK~@wHvBsHhFwOC[Wa@sGcEGI@w@jB}GxK}[v@_C~@gBzBuGbFwOl@kCj@_BMg@}B_BEMLk@tCkI|KuWr@u@nCo@h@e@`KeVhGyJ`CwEGc@oHqI_BqBCYrKqNnB{BBGIEm@`ADb@bH|Jt@`@~GAnDsB|@w@P]?iAi@aCcD{CsBsDa@sABc@b@y@B[iAyA[MCQpAaBN_@v@k@n@f@x@lAd@JxB}@hBSj@UdAuA`@cCXWl@Bz@bAn@Fz@[|@aBbBa@V]Dg@EsCR_@z@[Ja@Q}CDoAJc@r@m@xD{Bx@QOKFSlDiCr@qAd@sCZu@\FnBlEbArCLx@^Pv@bAj@J~Bo@bBdAf@?|@gApBeArBqB|CeCfA_BxBkA`CgCzCcCvAy@Zk@@a@{@eBGi@BMdAq@rApCj@`BnBnDCRs@f@KVx@nBtAvEd@bAf@j@l@^f@H|ABrCeAnCSdANfDzAh@D|CuAvA_ApDsAbJuCrFuAl@[~D}Db@?RNNz@PVpBVj@EXYp@_BRBd@^NMZmAzAcD|D_KfHyO|C_Ln@cBlDxBnAh@rANbGA~BT~Bh@nDxArCpBvD~D`B~BlBfE|@dDB\GNq@z@mAr@iDbBcCfCgCrDuAlCwBvGq@dEgCpTuEvTyBxIsI~_@mAfDuEzIy@tBUzAHtDaAVkABmBIyB_@Yp@c@xBa@x@iExDYEkAwBU?c@V}YdUSb@ClDMf@oFrE}H~FiPwBkDB}AOeQmFoFwBi@J_BhAmATyNe@u@c@{@kAi@cCSIC^\zA|AxCETq@t@]v@SpAGxCqArD

In [7]:
my_dataset

[{'resource_state': 2,
  'athlete': {'id': 644338, 'resource_state': 1},
  'name': 'Windy! 💨 ',
  'distance': 30998.3,
  'moving_time': 4379,
  'elapsed_time': 4714,
  'total_elevation_gain': 81.0,
  'type': 'Ride',
  'workout_type': 10,
  'id': 6937664224,
  'start_date': '2022-04-05T22:16:22Z',
  'start_date_local': '2022-04-05T15:16:22Z',
  'timezone': '(GMT-08:00) America/Los_Angeles',
  'utc_offset': -25200.0,
  'location_city': None,
  'location_state': None,
  'location_country': 'United States',
  'achievement_count': 14,
  'kudos_count': 6,
  'comment_count': 0,
  'athlete_count': 1,
  'photo_count': 0,
  'map': {'id': 'a6937664224',
   'summary_polyline': 'okseFt{hiVx@y@dBy@v@{@\\}@tBsHb@{@nBgBzBaBh@QbCSh@DTK~AsF|AuAh@q@n@oB`@_EDaDQcFFk@dD@b@IZeCbAyB`DgDvByAV?`Ap@x@NhEFb@SRsEPeK~@wHvBsHhFwOC[Wa@sGcEGI@w@jB}GxK}[v@_C~@gBzBuGbFwOl@kCj@_BMg@}B_BEMLk@tCkI|KuWr@u@nCo@h@e@`KeVhGyJ`CwEGc@oHqI_BqBCYrKqNnB{BBGIEm@`ADb@bH|Jt@`@~GAnDsB|@w@P]?iAi@aCcD{CsBsDa@sABc@b@y@B[iAyA[MCQpAaBN_@v@k

In [148]:
cols_to_use = ['Activity ID', 'Activity Date', 'Activity Type', 'Elapsed Time', 'Distance', 'Max Heart Rate',
       'Relative Effort', 'Commute', 'Activity Gear', 'Moving Time',
       'Distance.1', 'Max Speed', 'Average Speed', 'Elevation Gain',
       'Elevation Loss', 'Elevation Low', 'Elevation High', 'Max Grade',
       'Average Grade', 'Average Positive Grade', 'Average Negative Grade',
       'Max Cadence', 'Average Cadence', 'Max Heart Rate.1',
       'Average Heart Rate', 'Max Watts', 'Average Watts', 'Calories',
       'Max Temperature', 'Average Temperature',
       'Total Work', 'Uphill Time', 'Perceived Exertion',
       'Weighted Average Power', 'Power Count', 'Commute.1', 'Weather Temperature',
       'Dewpoint', 'Humidity', 'Weather Pressure', 'Wind Speed', 'Wind Gust', 'UV Index']

In [149]:
# Opening activities dataset

current_dir = os.getcwd() # Get current working directory
data_dir = 'data' # Name of the directory where the data is stored

activies_df = pd.read_csv(os.path.join(current_dir, data_dir,'activities.csv'), usecols=cols_to_use)
activies_df.tail()

,Activity ID,Activity Date,Activity Type,Elapsed Time,Distance,Max Heart Rate,Relative Effort,Commute,Activity Gear,Moving Time,...,Weighted Average Power,Power Count,Commute.1,Weather Temperature,Dewpoint,Humidity,Weather Pressure,Wind Speed,Wind Gust,UV Index
701,6873134182,"Mar 24, 2022, 2:26:44 AM",Workout,1240,0.00,126.0,3.0,False,NaN,1240.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
702,6878410847,"Mar 25, 2022, 3:01:05 AM",Workout,1819,0.00,170.0,49.0,False,NaN,1819.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
703,6883067397,"Mar 26, 2022, 12:31:44 AM",Workout,1228,0.00,120.0,3.0,False,NaN,1228.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
704,6888126531,"Mar 26, 2022, 10:42:55 PM",Ride,2892,19.40,177.0,70.0,False,Hillius Maximus,2639.0,...,148.0,2893.0,0.0,16.030001,9.99,0.67,1012.200012,2.56,5.40,4.0
705,6893936573,"Mar 27, 2022, 8:21:34 PM",Ride,6808,40.44,181.0,160.0,False,Hillius Maximus,6200.0,...,149.0,6809.0,0.0,14.520000,8.88,0.69,1007.500000,2.08,4.73,4.0


In [150]:
# Converting dates to datetime and extracting year, month, day
activies_df['Activity Date'] = pd.to_datetime(activies_df['Activity Date'])
activies_df['Year'] = activies_df['Activity Date'].dt.year
activies_df['Month'] = activies_df['Activity Date'].dt.month
activies_df['Weekday'] = activies_df['Activity Date'].dt.weekday
activies_df.tail()


,Activity ID,Activity Date,Activity Type,Elapsed Time,Distance,Max Heart Rate,Relative Effort,Commute,Activity Gear,Moving Time,...,Weather Temperature,Dewpoint,Humidity,Weather Pressure,Wind Speed,Wind Gust,UV Index,Year,Month,Weekday
701,6873134182,2022-03-24 02:26:44,Workout,1240,0.00,126.0,3.0,False,NaN,1240.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,3,3
702,6878410847,2022-03-25 03:01:05,Workout,1819,0.00,170.0,49.0,False,NaN,1819.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,3,4
703,6883067397,2022-03-26 00:31:44,Workout,1228,0.00,120.0,3.0,False,NaN,1228.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,3,5
704,6888126531,2022-03-26 22:42:55,Ride,2892,19.40,177.0,70.0,False,Hillius Maximus,2639.0,...,16.030001,9.99,0.67,1012.200012,2.56,5.40,4.0,2022,3,5
705,6893936573,2022-03-27 20:21:34,Ride,6808,40.44,181.0,160.0,False,Hillius Maximus,6200.0,...,14.520000,8.88,0.69,1007.500000,2.08,4.73,4.0,2022,3,6


In [139]:
# Grouping by activity type
grouped_by_type = activies_df.groupby(['Activity Type', 'Year', 'Month']).agg({'Activity ID': 'count', 'Distance':'sum', 'Elapsed Time':'sum', 'Calories':'sum'}).reset_index()
rides = grouped_by_type[grouped_by_type['Activity Type'] == 'Ride']
rides.head()

,Activity Type,Year,Month,Activity ID,Distance,Elapsed Time,Calories
6,Ride,2012,5,2,32.65,11818,490.022797
7,Ride,2012,6,12,246.64,71793,3750.283401
8,Ride,2012,7,8,277.39,73202,4948.270416
9,Ride,2012,8,10,465.10,157667,6788.660160
10,Ride,2012,9,5,378.92,113879,5480.945953


In [146]:
# Calculating cummulative sum of distance and elapsed time
rides['Cummulative Distance'] = rides.groupby(['Year']).agg({'Distance': 'cumsum'})
rides['Cummulative Calories'] = rides.groupby(['Year']).agg({'Calories': 'cumsum'})
rides.tail()

/var/folders/6t/_k6kmr152jq69rjlrcg91mb00000gn/T/ipykernel_63339/2027743540.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/6t/_k6kmr152jq69rjlrcg91mb00000gn/T/ipykernel_63339/2027743540.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Activity Type,Year,Month,Activity ID,Distance,Elapsed Time,Calories,Cummulative Annual Distance,Cummulative Distance,Cummulative Calories
80,Ride,2021,9,13,362.62,83039,9230.464172,3347.94,3347.94,81527.024940
81,Ride,2021,10,5,205.76,49198,4977.600159,3553.70,3553.70,86504.625099
82,Ride,2021,11,2,24.79,6344,596.000000,3578.49,3578.49,87100.625099
83,Ride,2022,2,1,28.02,4905,588.055115,28.02,28.02,588.055115
84,Ride,2022,3,16,350.88,71212,7999.356537,378.90,378.90,8587.411652


In [151]:
fig = px.line(rides, x="Month", y="Cummulative Distance", color='Year', markers=True)
fig.show()

In [159]:
grouped_by_bike = activies_df.groupby('Activity Gear').agg({'Distance':'sum'}).sort_values(by='Distance', ascending=False).reset_index()
grouped_by_bike

,Activity Gear,Distance
0,Caadie,9231.28
1,TIE Fighter,5259.49
2,Hillius Maximus,2870.27
3,The Sound,1417.10
4,Dirty McDirtBag,815.57
5,Hurt Enforcer,729.11
6,Red Barron,528.34
7,Purdie Little Thang,437.62
8,Big Boss Hoss,96.08


In [160]:
fig = px.bar(grouped_by_bike, x='Activity Gear', y='Distance', color='Activity Gear')
fig